## Preprocessing TEMPO satellite and ground data for model training

This is the jupyter notebook guide to merging and preparing data for gradient booster and random forest machine learning algorithm.


In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect(r"C:\Users\HP\Downloads\phase1_data (3).db")
cursor = conn.cursor()

# 1. Rename old table and create new one without 'humidity'
cursor.execute("""
    CREATE TABLE weather_data_new AS
    SELECT id, timestamp, temperature, wind_speed
    FROM weather_data
""")

# 2. Drop the old table
cursor.execute("DROP TABLE weather_data")

# 3. Rename new table to original name
cursor.execute("ALTER TABLE weather_data_new RENAME TO weather_data")

conn.commit()
conn.close()

print("✅ 'humidity' column deleted successfully from weather_data")


✅ 'humidity' column deleted successfully from weather_data


### Checking null data and preparing for imputation 

to imputate null data, checking how much of our data is null values


In [ ]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect(r"C:\Users\HP\Downloads\phase1_data (3).db")

# Load weather_data into pandas DataFrame
df = pd.read_sql("SELECT * FROM ground_data", conn)

conn.close()

# Calculate percentage of NULL values in each column
null_percentages = (df.isnull().sum() / len(df)) * 100

print("📊 Percentage of NULL values in each column:")
print(null_percentages)

📊 Percentage of NULL values in each column:
id            0.000000
timestamp     0.000000
latitude      0.000000
longitude     0.000000
pm25          2.282863
no2          37.720160
so2          51.356867
o3           67.420062
co           51.477814
dtype: float64


### Sorting data according to timestamps


In [ ]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect(r"C:\Users\HP\Downloads\phase1_data (2).db")

# Read and sort ground_data by timestamp
df_ground = pd.read_sql("SELECT * FROM ground_data ORDER BY timestamp ASC", conn)

conn.close()

# Show first few rows
print(df_ground.head())

# Save cleaned, sorted data to CSV
df_ground.to_csv("ground_data_sorted.csv", index=False)
print("✅ ground_data sorted by timestamp and saved to 'ground_data_sorted.csv'")


    id         timestamp  latitude  longitude  pm25   no2  so2     o3     co
0    1  2024-08-01 00:00  40.81600  -73.90200   6.9  16.3  NaN  0.028    NaN
1   32  2024-08-01 00:00  40.71961  -73.94771  16.1   NaN  NaN    NaN    NaN
2  112  2024-08-01 00:00  40.79970  -73.93432  13.5   NaN  NaN    NaN    NaN
3  128  2024-08-01 00:00  40.73614  -73.82153  10.5  20.1 -0.4  0.018  0.413
4  160  2024-08-01 00:00  40.73000  -73.98400  14.6   NaN  NaN    NaN    NaN
✅ ground_data sorted by timestamp and saved to 'ground_data_sorted.csv'


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"C:\Users\HP\Downloads\phase1_data (3).db")
df = pd.read_sql("SELECT * FROM ground_data LIMIT 20;", conn)
conn.close()
print(df)


    id         timestamp  latitude  longitude  pm25   no2  so2     o3    co
0    1  2023-04-03 00:00    40.816    -73.902   3.5  15.7  0.3  0.034  None
1    2  2023-04-06 00:00    40.816    -73.902   6.3  10.4  0.0  0.027  None
2    3  2023-04-09 00:00    40.816    -73.902   6.7  28.2  0.4  0.019  None
3    4  2023-04-12 00:00    40.816    -73.902   6.9  11.5  0.5  0.046  None
4    5  2023-04-15 00:00    40.816    -73.902   9.8  17.3  0.6  0.024  None
5    6  2023-04-18 00:00    40.816    -73.902   2.9   4.8  0.1  0.035  None
6    7  2023-04-21 00:00    40.816    -73.902   8.5  27.6  0.3  0.022  None
7    8  2023-04-24 00:00    40.816    -73.902   3.3   5.4  0.0  0.031  None
8    9  2023-04-27 00:00    40.816    -73.902  11.5  19.7  0.1  0.032  None
9   10  2023-04-30 00:00    40.816    -73.902   2.9   3.8  0.0  0.041  None
10  11  2023-05-03 00:00    40.816    -73.902   1.9   9.8  0.2  0.021  None
11  12  2023-05-06 00:00    40.816    -73.902   7.3  38.8  0.4  0.002  None
12  13  2023

### Interpolation of missing NaN values

Choosen method for interpolation includes:
Handling short gaps with linear interpolation polation + rolling mean, big gaps with forward fill, which imputates on colums like c02, no2, c0 and o3.


In [ ]:
import sqlite3
import pandas as pd
from pandas.tseries.frequencies import to_offset

# ----------------- USER PARAMETERS -----------------
DB_PATH = r"C:\Users\HP\Downloads\phase1_data (2).db"   # adjust if needed
TABLE = "ground_data"                                   # or "weather_data"
TS_COL = "timestamp"
OUT_CSV = "ground_data_all_imputed.csv"
short_gap_max = 3        # <= this many consecutive missing steps -> treat as short gap (interpolate)
rolling_window = 3       # smoothing window (centered rolling mean)
WRITE_BACK_TO_DB = False # set True to write cleaned table as "<TABLE>_imputed" back to DB
# ---------------------------------------------------

# --- load ---
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql(f"SELECT * FROM {TABLE}", conn)
conn.close()

# --- timestamp parsing & drop unparsable ---
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL]).copy()

# --- separate columns ---
all_cols = list(df.columns)
non_ts_cols = [c for c in all_cols if c != TS_COL]

# --- aggregate duplicate timestamps (mean for numeric cols, first() for others) ---
# numeric aggregation
numeric_cols = [c for c in non_ts_cols if pd.api.types.is_numeric_dtype(df[c])]
non_numeric_cols = [c for c in non_ts_cols if c not in numeric_cols]

# numeric mean
df_num = df.groupby(TS_COL, as_index=False)[numeric_cols].mean() if numeric_cols else pd.DataFrame({TS_COL: sorted(df[TS_COL].unique())})

# keep first non-numeric if present
if non_numeric_cols:
    df_first = df.groupby(TS_COL, as_index=False)[non_numeric_cols].first()
    df_agg = pd.merge(df_num, df_first, on=TS_COL, how="left")
else:
    df_agg = df_num

# --- sort & infer frequency ---
df_agg = df_agg.sort_values(TS_COL).reset_index(drop=True)

if df_agg[TS_COL].shape[0] >= 2:
    med_delta = df_agg[TS_COL].diff().dropna().median()
    try:
        freq = to_offset(med_delta)
    except Exception:
        freq = "H"
else:
    freq = "H"

# --- reindex to regular timeline ---
full_idx = pd.date_range(start=df_agg[TS_COL].min(), end=df_agg[TS_COL].max(), freq=freq)
df_reidx = df_agg.set_index(TS_COL).reindex(full_idx).rename_axis(TS_COL)

# --- record original missing (for diagnostics) ---
orig_missing_pct = (df_reidx.isna().sum() / len(df_reidx) * 100).round(4)

# --- Impute numeric columns ---
for col in numeric_cols:
    if col not in df_reidx.columns:
        continue
    s = pd.to_numeric(df_reidx[col], errors="coerce")

    # If ENTIRE column is NaN -> fill with median (or 0 fallback)
    if s.isna().all():
        med = s.median()
        if pd.isna(med):
            med = 0.0
        df_reidx[col] = med
        print(f"⚠️ Column '{col}' was completely empty — filled with median/fallback {med}")
        continue

    # 1) time interpolation limited to short gaps
    s_interp = s.interpolate(method="time", limit=short_gap_max, limit_direction="both")

    # 2) smoothing of interpolated values
    s_smooth = s_interp.rolling(window=rolling_window, min_periods=1, center=True).mean()

    # 3) fill long gaps with forward-fill, then backward-fill for leading NaNs
    s_filled = s_smooth.fillna(method="ffill").fillna(method="bfill")

    # 4) if anything still NaN (rare), fill with column median or 0
    if s_filled.isna().any():
        fallback = s_filled.median()
        if pd.isna(fallback):
            fallback = 0.0
        s_filled = s_filled.fillna(fallback)

    df_reidx[col] = s_filled

# --- Impute non-numeric columns ---
for col in non_numeric_cols:
    if col not in df_reidx.columns:
        continue
    s = df_reidx[col]

    if s.isna().all():
        df_reidx[col] = "missing"
        print(f"⚠️ Column '{col}' was completely empty — filled with 'missing'")
        continue

    # fill with mode (most frequent) if possible
    try:
        mode_val = s.mode(dropna=True)[0]
        df_reidx[col] = s.fillna(mode_val)
    except Exception:
        df_reidx[col] = s.fillna("missing")

# --- Final sanity: any remaining NaNs -> numeric 0, non-numeric 'missing' ---
for col in df_reidx.columns:
    if df_reidx[col].isna().any():
        if pd.api.types.is_numeric_dtype(df_reidx[col]):
            df_reidx[col] = df_reidx[col].fillna(0)
        else:
            df_reidx[col] = df_reidx[col].fillna("missing")

# --- Diagnostics ---
final_missing_pct = (df_reidx.isna().sum() / len(df_reidx) * 100).round(6)   # should be zero now
print("\nOriginal % missing (sample):")
print(orig_missing_pct[orig_missing_pct > 0].sort_values(ascending=False).head(20))
print("\nFinal % missing (should be 0 for all):")
print(final_missing_pct[final_missing_pct > 0])

# --- Save to CSV ---
df_out = df_reidx.reset_index()
df_out.to_csv(OUT_CSV, index=False)
print(f"\n✅ All imputations applied and saved to: {OUT_CSV}")

# --- Optional: write back to DB as new table ---
if WRITE_BACK_TO_DB:
    conn = sqlite3.connect(DB_PATH)
    new_table_name = f"{TABLE}_imputed"
    df_out.to_sql(new_table_name, conn, if_exists="replace", index=False)
    conn.close()
    print(f"✅ Cleaned table written back to DB as: {new_table_name}")



Original % missing (sample):
o3    0.0906
dtype: float64

Final % missing (should be 0 for all):
Series([], dtype: float64)

✅ All imputations applied and saved to: ground_data_all_imputed.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_23060\2875817492.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s_filled = s_smooth.fillna(method="ffill").fillna(method="bfill")
C:\Users\HP\AppData\Local\Temp\ipykernel_23060\2875817492.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s_filled = s_smooth.fillna(method="ffill").fillna(method="bfill")
C:\Users\HP\AppData\Local\Temp\ipykernel_23060\2875817492.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s_filled = s_smooth.fillna(method="ffill").fillna(method="bfill")
C:\Users\HP\AppData\Local\Temp\ipykernel_23060\2875817492.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s_

### Checking data after imputation


In [ ]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect(r"C:\Users\HP\Downloads\phase1_data (3).db")

# Load weather_data into pandas DataFrame
df = pd.read_sql("SELECT * FROM weather_data", conn)

conn.close()

# Calculate percentage of NULL values in each column
null_percentages = (df.isnull().sum() / len(df)) * 100

print("📊 Percentage of NULL values in each column:")
print(null_percentages)

📊 Percentage of NULL values in each column:
id             0.0
timestamp      0.0
temperature    0.0
wind_speed     0.0
dtype: float64


### making csv files for model training



In [ ]:
import sqlite3
import pandas as pd

# Load the cleaned CSV
df = pd.read_csv("gdf.csv", parse_dates=["timestamp"])

# Connect (this will create db file if it doesn’t exist)
conn = sqlite3.connect("gdf.db")   # you can name it anything you want

# Write DataFrame into database as a new table
df.to_sql("gdf_table", conn, if_exists="replace", index=False)

# Verify
print("✅ gdf.csv has been imported into gdf.db as table 'gdf_table'.")

# Optional: show first 5 rows from DB
print(pd.read_sql("SELECT * FROM gdf_table LIMIT 5", conn))

conn.close()


✅ gdf.csv has been imported into gdf.db as table 'gdf_table'.
             timestamp           id   latitude  longitude      pm25  \
0  2023-04-01 00:00:00  5649.333333  40.757242 -73.997764  9.233333   
1  2023-04-01 01:00:00  5650.333333  40.757242 -73.997764  9.100000   
2  2023-04-01 02:00:00  5651.333333  40.757242 -73.997764  8.811111   
3  2023-04-01 03:00:00  5652.333333  40.757242 -73.997764  9.133333   
4  2023-04-01 04:00:00  5653.333333  40.757242 -73.997764  9.650000   

         no2       so2      o3        co  
0  21.937500  0.566667  0.0270  0.336500  
1  20.191667  0.544444  0.0240  0.323000  
2  18.016667  0.488889  0.0300  0.297778  
3  17.733333  0.488889  0.0285  0.296333  
4  20.083333  0.477778  0.0280  0.308889  


In [ ]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect(r'C:\Users\HP\Desktop\suraj\gdf.db')

# Load weather_data into pandas DataFrame
df = pd.read_sql("SELECT * FROM gdf_table ", conn)

conn.close()

# Calculate percentage of NULL values in each column
null_percentages = (df.isnull().sum() / len(df)) * 100

print("📊 Percentage of NULL values in each column:")
print(null_percentages)

📊 Percentage of NULL values in each column:
timestamp    0.0
id           0.0
latitude     0.0
longitude    0.0
pm25         0.0
no2          0.0
so2          0.0
o3           0.0
co           0.0
dtype: float64


## Putting data in database


In [ ]:
import sqlite3
import pandas as pd

# Path to your database
DB_PATH = r'C:\Users\HP\Downloads\phase1_data (3).db'

# Connect to database
conn = sqlite3.connect(DB_PATH)

# Load weather_data into a DataFrame
df_weather = pd.read_sql("SELECT * FROM weather_data", conn)

conn.close()

# Save as CSV
df_weather.to_csv("wdf.csv", index=False)

print("✅ Exported weather_data table from phase1_data3.db to wdf.csv")


✅ Exported weather_data table from phase1_data3.db to wdf.csv


## Preprocessing ground weather data



In [ ]:
import sqlite3
import pandas as pd

# Load weather data CSV
wdf = pd.read_csv("wdf.csv", parse_dates=["timestamp"])

# Connect to existing gdf.db
conn = sqlite3.connect("gdf.db")

# Write DataFrame into database as a new table
wdf.to_sql("wdf_table", conn, if_exists="replace", index=False)

print("✅ wdf.csv has been imported into gdf.db as table 'wdf_table'.")

# Optional: check first few rows
print(pd.read_sql("SELECT * FROM wdf_table LIMIT 5", conn))

conn.close()


✅ wdf.csv has been imported into gdf.db as table 'wdf_table'.
   id            timestamp  temperature  wind_speed
0   1  2023-04-01 00:00:00         16.6        22.7
1   2  2023-04-02 00:00:00          9.9        27.7
2   3  2023-04-03 00:00:00          9.4        12.6
3   4  2023-04-04 00:00:00         15.0         6.5
4   5  2023-04-05 00:00:00         13.2        12.6


### Merging satellite data with ground data

In [ ]:
import sqlite3
import pandas as pd

DB = "gdf.db"             # your DB file
G_TABLE = "gdf_table"     # hourly ground table
W_TABLE = "wdf_table"     # daily weather table

# load
conn = sqlite3.connect(DB)
gdf = pd.read_sql(f"SELECT * FROM {G_TABLE}", conn)
wdf = pd.read_sql(f"SELECT * FROM {W_TABLE}", conn)
conn.close()

# parse timestamps robustly
gdf['timestamp'] = pd.to_datetime(gdf['timestamp'], errors='coerce')
wdf['timestamp'] = pd.to_datetime(wdf['timestamp'], errors='coerce')

# drop rows with bad timestamps
gdf = gdf.dropna(subset=['timestamp']).copy()
wdf = wdf.dropna(subset=['timestamp']).copy()

# create a calendar date column (YYYY-MM-DD) for merging
gdf['date'] = gdf['timestamp'].dt.date
wdf['date'] = wdf['timestamp'].dt.date

# if weather table contains multiple rows per date, aggregate it (mean for numeric)
num_cols = wdf.select_dtypes(include='number').columns.tolist()
if len(wdf.groupby('date')) > len(wdf):
    aggs = {c: 'mean' for c in num_cols}
    wdf_daily = wdf.groupby('date', as_index=False).agg(aggs)
    # keep non-numeric columns if necessary (first)
    for c in wdf.columns:
        if c not in num_cols + ['timestamp', 'date']:
            wdf_daily[c] = wdf.groupby('date')[c].first().values
else:
    # reduce to numeric + date
    wdf_daily = wdf.drop(columns=[c for c in wdf.columns if c not in num_cols + ['date']], errors='ignore')

# merge: broadcast daily weather to each hourly ground row with same date
merged_hourly = pd.merge(gdf, wdf_daily, on='date', how='left', suffixes=('_ground', '_weather'))

# optional: drop the helper 'date' column (or keep it)
# merged_hourly = merged_hourly.drop(columns=['date'])

# Save
merged_hourly.to_csv("merged_hourly_ground_with_daily_weather.csv", index=False)
print("Saved: merged_hourly_ground_with_daily_weather.csv")

# Optional: write back to DB
conn = sqlite3.connect(DB)
merged_hourly.to_sql("merged_hourly_ground_weather", conn, if_exists="replace", index=False)
conn.close()
print("Wrote merged table to DB as 'merged_hourly_ground_weather'")


Saved: merged_hourly_ground_with_daily_weather.csv
Wrote merged table to DB as 'merged_hourly_ground_weather'


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"C:\Users\HP\Desktop\suraj\gdf.db")
df = pd.read_sql("SELECT * FROM ground_data LIMIT 20;", conn)
conn.close()
print(df)


In [ ]:
import sqlite3
import pandas as pd

# Path to your merged CSV
MERGED_CSV = "merged_hourly_ground_with_daily_weather.csv"   # or merged_daily_ground_weather.csv

# Load the CSV
df = pd.read_csv(MERGED_CSV, parse_dates=["timestamp"], low_memory=False)

# Connect to gdf.db
conn = sqlite3.connect("gdf.db")

# Write as a new table inside gdf.db
df.to_sql("merged_table", conn, if_exists="replace", index=False)

conn.close()

print("✅ Merged CSV imported into gdf.db as table 'merged_table'")


✅ Merged CSV imported into gdf.db as table 'merged_table'


## FEATURE ENGINEERING

engineer_gdf_features.py

Reads gdf.db -> merged_table and creates features:
- hour, dayofweek, is_weekend
- lag features for pollutant columns (grouped by latitude+longitude)
- no2_wind = no2 * windspeed
- pm25_temp = pm25 / temperature (safe for zero temps)
- total_pollutants = no2 + so2 + co + pm25

Outputs:
- CSV: merged_table_engineered.csv
- New DB table: merged_table_engineered

Edit DB_PATH and other config below if required.

In [ ]:

import os
import sqlite3
import numpy as np
import pandas as pd

# ---------- CONFIG ----------
DB_PATH = r"C:\Users\HP\Desktop\suraj\gdf.db"   # path to your gdf.db
SRC_TABLE = "merged_table"
OUT_CSV = r"merged_table_engineered.csv"
OUT_TABLE = "merged_table_engineered"

# Lags to generate (in rows). If data is hourly, 1 means previous hour (previous row).
LAGS = [1, 3, 6, 24]

# Which pollutant columns to create lag features for (will skip if column missing)
LAG_COLS = ["no2", "pm25", "so2", "co", "o3", "total_pollutants"]

# Rolling window sizes (in rows) to create rolling mean features
ROLL_WINDOWS = [3, 6, 24]
# ----------------------------

def read_db_table(db_path, table_name):
    if not os.path.isfile(db_path):
        raise FileNotFoundError(f"DB not found: {db_path}")
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f'SELECT * FROM "{table_name}";', conn)
    conn.close()
    return df

def write_table_to_db(df, db_path, table_name, if_exists='replace'):
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)
    conn.close()

def safe_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

def main():
    print("Loading table:", SRC_TABLE)
    df = read_db_table(DB_PATH, SRC_TABLE)
    if df.empty:
        raise SystemExit("Source table is empty or not found.")

    # normalize column names (rename pm2.5 -> pm25)
    if "pm2.5" in df.columns:
        df = df.rename(columns={"pm2.5": "pm25"})
    # also rename e.g. PM25 or pm25 variations to lower-case consistent names
    df.columns = [c.strip() for c in df.columns]

    # choose timestamp column: prefer 'timestamp' then 'date'
    ts_col = None
    for c in df.columns:
        if c.lower() == "timestamp":
            ts_col = c
            break
    if ts_col is None:
        for c in df.columns:
            if c.lower() == "date":
                ts_col = c
                break
    if ts_col is None:
        raise SystemExit("No timestamp/date column found (expected 'timestamp' or 'date').")

    # parse timestamp
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    if df[ts_col].isna().all():
        raise SystemExit(f"All values in {ts_col} failed to parse as datetime.")

    # create datetime features
    df = df.sort_values(by=[ts_col]).reset_index(drop=True)
    df["hour"] = df[ts_col].dt.hour
    df["dayofweek"] = df[ts_col].dt.dayofweek
    df["is_weekend"] = df["dayofweek"].isin([5,6]).astype(int)  # 5=Sat, 6=Sun

    # ensure numeric for key columns
    numeric_candidates = ["no2", "so2", "co", "pm25", "o3", "temperature", "windspeed"]
    df = safe_numeric(df, numeric_candidates)

    # create total_pollutants = NO2 + SO2 + CO + PM25 (only sums available cols)
    pollutants_for_sum = [c for c in ["no2","so2","co","pm25"] if c in df.columns]
    if pollutants_for_sum:
        df["total_pollutants"] = df[pollutants_for_sum].sum(axis=1, skipna=True)
    else:
        df["total_pollutants"] = np.nan

    # interaction features
    if "no2" in df.columns and "windspeed" in df.columns:
        df["no2_wind"] = df["no2"] * df["windspeed"]
    else:
        df["no2_wind"] = np.nan

    # pm25_temp = pm25 / temperature (avoid division by zero)
    if "pm25" in df.columns and "temperature" in df.columns:
        temp = df["temperature"].replace({0: np.nan})  # avoid div by zero
        df["pm25_temp"] = df["pm25"] / temp
    else:
        df["pm25_temp"] = np.nan

    # prepare for lagging: sort by location (if present) and timestamp
    group_cols = []
    if "latitude" in df.columns and "longitude" in df.columns:
        group_cols = ["latitude", "longitude"]
        sort_cols = group_cols + [ts_col]
    else:
        sort_cols = [ts_col]

    df = df.sort_values(by=sort_cols).reset_index(drop=True)

    # create lag features grouped by location if available, else global
    print("Creating lag features grouped by:", group_cols if group_cols else "GLOBAL")
    for col in LAG_COLS:
        if col not in df.columns:
            print(f" - skipping lag for missing column: {col}")
            continue
        # ensure numeric
        df[col] = pd.to_numeric(df[col], errors="coerce")
        for lag in LAGS:
            lag_name = f"{col}_lag{lag}"
            if group_cols:
                df[lag_name] = df.groupby(group_cols)[col].shift(lag)
            else:
                df[lag_name] = df[col].shift(lag)
        # difference to previous
        diff_name = f"{col}_diff_lag1"
        if group_cols:
            df[diff_name] = df[col] - df.groupby(group_cols)[col].shift(1)
        else:
            df[diff_name] = df[col] - df[col].shift(1)

        # rolling means (transform keeps index alignment)
        for w in ROLL_WINDOWS:
            roll_name = f"{col}_roll_mean_{w}"
            if group_cols:
                df[roll_name] = df.groupby(group_cols)[col].transform(lambda s: s.rolling(window=w, min_periods=1).mean())
            else:
                df[roll_name] = df[col].rolling(window=w, min_periods=1).mean()

    # optional: keep NaNs for first few rows per group (they indicate lack of history)
    # You can fill them later for modeling: e.g., df.fillna(method='ffill') or df.fillna(0)

    # Save outputs
    print("Saving CSV to:", OUT_CSV)
    df.to_csv(OUT_CSV, index=False)

    # Write engineered table back to DB (new table, do not overwrite original)
    print("Writing engineered table to DB as:", OUT_TABLE)
    write_table_to_db(df, DB_PATH, OUT_TABLE, if_exists='replace')

    # Quick preview
    print("Done. Preview of engineered columns:")
    print(df.head(5).T.head(30))

if __name__ == "__main__":
    main()


Loading table: merged_table
Creating lag features grouped by: ['latitude', 'longitude']
Saving CSV to: merged_table_engineered.csv
Writing engineered table to DB as: merged_table_engineered
Done. Preview of engineered columns:
                                    0                    1  \
timestamp         2023-04-06 00:00:00  2023-04-12 00:00:00   
id_ground                 1077.083333          1102.416667   
latitude                     40.75275             40.75275   
longitude                  -73.973799           -73.973799   
pm25                           10.725             9.177778   
no2                         16.688889            13.188889   
so2                             0.375             0.783333   
o3                              0.028                0.047   
co                           0.266578               0.1884   
date                       2023-04-06           2023-04-12   
id_weather                          6                   12   
temperature                  

In [ ]:
#!/usr/bin/env python3
"""
show_merged_engineered_nulls.py

Connects to gfd.db and displays NULL/empty-string counts + percentages
for every column in the merged_engineered table. Also prints sample rows
that contain any NULL/empty value.

No CSV is written — output is displayed to the console only.

Edit DB_PATH if your DB is in a different folder.
"""
import os
import sqlite3
import sys

# === EDIT IF NEEDED ===
DB_PATH = r"C:\Users\HP\Desktop\suraj\gdf.db"                 # full path if needed, e.g. r"C:\Users\HP\Downloads\gfd.db"
TABLE = "merged_table_engineered"
SAMPLE_ROWS_TO_SHOW = 10
# ======================

def get_columns(conn, table):
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info('{table}');")
    return [row[1] for row in cur.fetchall()]

def total_rows(conn, table):
    cur = conn.cursor()
    cur.execute(f"SELECT COUNT(*) FROM '{table}';")
    return cur.fetchone()[0]

def count_null_and_empty(conn, table, col):
    cur = conn.cursor()
    cur.execute(f"SELECT COUNT(*) FROM '{table}' WHERE \"{col}\" IS NULL;")
    null_count = cur.fetchone()[0]
    cur.execute(f"SELECT COUNT(*) FROM '{table}' WHERE TRIM(CAST(\"{col}\" AS TEXT)) = '' AND \"{col}\" IS NOT NULL;")
    empty_count = cur.fetchone()[0]
    return null_count, empty_count

def sample_rows_with_any_empty(conn, table, cols, limit=10):
    cur = conn.cursor()
    conds = []
    for c in cols:
        conds.append(f"\"{c}\" IS NULL")
        conds.append(f"TRIM(CAST(\"{c}\" AS TEXT)) = ''")
    where_clause = " OR ".join(conds)
    sql = f"SELECT * FROM '{table}' WHERE {where_clause} LIMIT {limit};"
    cur.execute(sql)
    rows = cur.fetchall()
    headers = [d[0] for d in cur.description] if cur.description else []
    return headers, rows

def main():
    if not os.path.isfile(DB_PATH):
        print("ERROR: DB file not found at:", DB_PATH)
        sys.exit(1)

    conn = sqlite3.connect(DB_PATH)
    cols = get_columns(conn, TABLE)
    if not cols:
        print(f"ERROR: Table '{TABLE}' not found or has no columns in {DB_PATH}")
        conn.close()
        sys.exit(1)

    total = total_rows(conn, TABLE)
    if total == 0:
        print(f"Table '{TABLE}' has 0 rows.")
        conn.close()
        sys.exit(0)

    report = []
    for c in cols:
        null_count, empty_count = count_null_and_empty(conn, TABLE, c)
        total_empty = null_count + empty_count
        pct_null = round(100.0 * null_count / total, 6)
        pct_total_empty = round(100.0 * total_empty / total, 6)
        report.append({
            "column": c,
            "null_count": null_count,
            "empty_string_count": empty_count,
            "total_empty_count": total_empty,
            "pct_null_%": pct_null,
            "pct_total_empty_%": pct_total_empty
        })

    # sort by pct_total_empty descending
    report_sorted = sorted(report, key=lambda r: r["pct_total_empty_%"], reverse=True)

    # print summary
    print(f"\nTable: {TABLE}  |  Total rows: {total}\n")
    print(f"{'column':40} {'nulls':>8} {'empties':>8} {'total_empty':>12} {'%null':>9} {'%total':>9}")
    print("-" * 100)
    for r in report_sorted:
        print(f"{r['column']:<40} {r['null_count']:>8} {r['empty_string_count']:>8} {r['total_empty_count']:>12} {r['pct_null_%']:>8}% {r['pct_total_empty_%']:>8}%")

    # show sample rows containing any empty value
    print(f"\nShowing up to {SAMPLE_ROWS_TO_SHOW} sample rows that contain any NULL or empty-string value:")
    headers, rows = sample_rows_with_any_empty(conn, TABLE, cols, limit=SAMPLE_ROWS_TO_SHOW)
    if rows:
        # print header
        print("\n" + " | ".join(headers))
        for r in rows:
            # truncate long fields for console readability
            print(" | ".join((str(v)[:120] if v is not None else "NULL") for v in r))
    else:
        print("No sample rows with empties found (first", SAMPLE_ROWS_TO_SHOW, "rows).")

    conn.close()

if __name__ == "__main__":
    main()



Table: merged_table_engineered  |  Total rows: 2184

column                                      nulls  empties  total_empty     %null    %total
----------------------------------------------------------------------------------------------------
no2_wind                                     2184        0         2184    100.0%    100.0%
no2_lag24                                      79        0           79 3.617216% 3.617216%
pm25_lag24                                     79        0           79 3.617216% 3.617216%
so2_lag24                                      79        0           79 3.617216% 3.617216%
co_lag24                                       79        0           79 3.617216% 3.617216%
o3_lag24                                       79        0           79 3.617216% 3.617216%
total_pollutants_lag24                         79        0           79 3.617216% 3.617216%
no2_lag6                                       25        0           25 1.144689% 1.144689%
pm25_lag6        

### impute_and_drop_no2_wind.py

- Loads gfd.db -> merged_engineered
- Drops column 'no2_wind' if present
- Imputes numeric columns with median
- Imputes object/string columns with mode (or '' if no mode)
- Backs up original table, writes imputed table replacing merged_engineered
- Saves a CSV copy merged_engineered_imputed.csv for inspection

Edit DB_PATH/TABLE if needed.


In [ ]:
#!/usr/bin/env python3

import os
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime

# ---- CONFIG ----
DB_PATH = r"C:\Users\HP\Desktop\suraj\gdf.db"                       # path to your DB
TABLE = "merged_table_engineered"               # table to modify
IMPUTED_TABLE = TABLE                     # we will replace TABLE after backup
CSV_OUT = "merged_engineered_imputed.csv" # local CSV export for inspection
# ----------------

def read_table(db_path, table):
    if not os.path.isfile(db_path):
        raise FileNotFoundError(f"DB not found: {db_path}")
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f'SELECT * FROM "{table}";', conn)
    conn.close()
    return df

def write_table(db_path, table, df, if_exists='replace'):
    conn = sqlite3.connect(db_path)
    df.to_sql(table, conn, if_exists=if_exists, index=False)
    conn.close()

def backup_table(db_path, table):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_name = f"{table}_backup_{ts}"
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # if backup already exists (very unlikely), drop it
    cur.execute(f"DROP TABLE IF EXISTS \"{backup_name}\";")
    # create backup as copy
    cur.execute(f'CREATE TABLE "{backup_name}" AS SELECT * FROM "{table}";')
    conn.commit()
    conn.close()
    return backup_name

def is_datetime_series(s: pd.Series, sample_n=50):
    # heuristics: pandas dtype datetime, or many values parseable to datetimes
    if np.issubdtype(s.dtype, np.datetime64):
        return True
    # check sample
    non_null = s.dropna().astype(str).head(sample_n)
    if len(non_null) == 0:
        return False
    parseable = 0
    for v in non_null:
        try:
            pd.to_datetime(v)
            parseable += 1
        except Exception:
            pass
    return (parseable / len(non_null)) > 0.6

def main():
    print("Loading table:", TABLE, "from DB:", DB_PATH)
    df = read_table(DB_PATH, TABLE)
    if df.empty:
        print("Table is empty or not found. Exiting.")
        return

    print("Original shape:", df.shape)
    # normalize column names (strip)
    df.columns = [c.strip() for c in df.columns]

    # 1) Drop no2_wind if exists
    if "no2_wind" in df.columns:
        df = df.drop(columns=["no2_wind"])
        print("Dropped column: no2_wind")
    else:
        print("Column 'no2_wind' not present — nothing to drop.")

    # 2) Detect column types and plan imputations
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    object_cols = df.select_dtypes(include=['object']).columns.tolist()
    # Also detect datetime-like columns among object columns
    datetime_cols = [c for c in df.columns if is_datetime_series(df[c])]
    # Remove detected datetime columns from object_cols (don't impute them as categoricals)
    object_cols = [c for c in object_cols if c not in datetime_cols]

    print("Numeric columns:", numeric_cols)
    print("Datetime-like columns:", datetime_cols)
    print("Object/categorical columns to impute with mode:", object_cols)

    # 3) Impute numeric cols with median
    for c in numeric_cols:
        median_val = df[c].median(skipna=True)
        if pd.isna(median_val):
            # if median cannot be computed (all NaN), fill with 0
            median_val = 0
        df[c] = df[c].fillna(median_val)
        print(f"Imputed numeric column '{c}' with median = {median_val}")

    # 4) Impute object/categorical columns with mode (most frequent)
    for c in object_cols:
        try:
            mode_series = df[c].mode(dropna=True)
            if not mode_series.empty:
                mode_val = mode_series.iloc[0]
            else:
                mode_val = ""
        except Exception:
            mode_val = ""
        df[c] = df[c].fillna(mode_val)
        # also replace empty strings consisting of only whitespace with mode_val
        df[c] = df[c].replace(r'^\s*$', mode_val, regex=True)
        print(f"Imputed object column '{c}' with mode (or '') = {repr(mode_val)}")

    # 5) For datetime columns: leave as is (but try to parse to datetime dtype)
    for c in datetime_cols:
        df[c] = pd.to_datetime(df[c], errors="coerce")
        print(f"Parsed datetime-like column '{c}' to datetime dtype (NaT if parse fails)")

    # 6) Backup original table in DB then replace with imputed
    print("Creating backup of original table in DB...")
    backup_name = backup_table(DB_PATH, TABLE)
    print("Backup table created:", backup_name)

    print("Writing imputed table back to DB, replacing original table...")
    write_table(DB_PATH, TABLE, df, if_exists='replace')
    print("Replaced table:", TABLE)

    # 7) Save local CSV for quick inspection (optional, user said earlier not necessary; left here for convenience)
    try:
        df.to_csv(CSV_OUT, index=False)
        print("Saved imputed CSV to:", CSV_OUT)
    except Exception as e:
        print("Could not save CSV (continuing):", e)

    print("Done. Imputation complete. Original backed up as:", backup_name)
    print("Final shape:", df.shape)

if __name__ == "__main__":
    main()


Loading table: merged_table_engineered from DB: C:\Users\HP\Desktop\suraj\gdf.db
Original shape: (2184, 67)
Dropped column: no2_wind
Numeric columns: ['id_ground', 'latitude', 'longitude', 'pm25', 'no2', 'so2', 'o3', 'co', 'id_weather', 'temperature', 'wind_speed', 'hour', 'dayofweek', 'is_weekend', 'total_pollutants', 'pm25_temp', 'no2_lag1', 'no2_lag3', 'no2_lag6', 'no2_lag24', 'no2_diff_lag1', 'no2_roll_mean_3', 'no2_roll_mean_6', 'no2_roll_mean_24', 'pm25_lag1', 'pm25_lag3', 'pm25_lag6', 'pm25_lag24', 'pm25_diff_lag1', 'pm25_roll_mean_3', 'pm25_roll_mean_6', 'pm25_roll_mean_24', 'so2_lag1', 'so2_lag3', 'so2_lag6', 'so2_lag24', 'so2_diff_lag1', 'so2_roll_mean_3', 'so2_roll_mean_6', 'so2_roll_mean_24', 'co_lag1', 'co_lag3', 'co_lag6', 'co_lag24', 'co_diff_lag1', 'co_roll_mean_3', 'co_roll_mean_6', 'co_roll_mean_24', 'o3_lag1', 'o3_lag3', 'o3_lag6', 'o3_lag24', 'o3_diff_lag1', 'o3_roll_mean_3', 'o3_roll_mean_6', 'o3_roll_mean_24', 'total_pollutants_lag1', 'total_pollutants_lag3', 'tot

### Puting final table in database and preparing training csv


In [ ]:
import sqlite3
import pandas as pd
import os

# === CONFIG ===
CSV_PATH = r"C:\Users\HP\Desktop\suraj\merged_engineered_imputed.csv"    # path to your CSV
DB_PATH = r"C:\Users\HP\Desktop\suraj\gdf.db"             # path to your SQLite DB
TABLE_NAME = "final_table"      # name of table to create/replace
# =================

def main():
    if not os.path.isfile(CSV_PATH):
        raise FileNotFoundError(f"CSV not found: {CSV_PATH}")
    if not os.path.isfile(DB_PATH):
        print(f"WARNING: DB not found, creating new one at {DB_PATH}")

    # Load CSV into DataFrame
    df = pd.read_csv(CSV_PATH)

    print("CSV loaded, shape:", df.shape)
    print("Columns:", df.columns.tolist())

    # Connect to DB
    conn = sqlite3.connect(DB_PATH)

    # Write DataFrame to SQLite (replace if table already exists)
    df.to_sql(TABLE_NAME, conn, if_exists="replace", index=False)

    conn.close()
    print(f"✅ Table '{TABLE_NAME}' written to {DB_PATH}")

if __name__ == "__main__":
    main()


CSV loaded, shape: (2184, 66)
Columns: ['timestamp', 'id_ground', 'latitude', 'longitude', 'pm25', 'no2', 'so2', 'o3', 'co', 'date', 'id_weather', 'temperature', 'wind_speed', 'hour', 'dayofweek', 'is_weekend', 'total_pollutants', 'pm25_temp', 'no2_lag1', 'no2_lag3', 'no2_lag6', 'no2_lag24', 'no2_diff_lag1', 'no2_roll_mean_3', 'no2_roll_mean_6', 'no2_roll_mean_24', 'pm25_lag1', 'pm25_lag3', 'pm25_lag6', 'pm25_lag24', 'pm25_diff_lag1', 'pm25_roll_mean_3', 'pm25_roll_mean_6', 'pm25_roll_mean_24', 'so2_lag1', 'so2_lag3', 'so2_lag6', 'so2_lag24', 'so2_diff_lag1', 'so2_roll_mean_3', 'so2_roll_mean_6', 'so2_roll_mean_24', 'co_lag1', 'co_lag3', 'co_lag6', 'co_lag24', 'co_diff_lag1', 'co_roll_mean_3', 'co_roll_mean_6', 'co_roll_mean_24', 'o3_lag1', 'o3_lag3', 'o3_lag6', 'o3_lag24', 'o3_diff_lag1', 'o3_roll_mean_3', 'o3_roll_mean_6', 'o3_roll_mean_24', 'total_pollutants_lag1', 'total_pollutants_lag3', 'total_pollutants_lag6', 'total_pollutants_lag24', 'total_pollutants_diff_lag1', 'total_pollut

In [ ]:
import pandas as pd
import sqlite3

CSV_PATH = r"C:\Users\HP\Desktop\suraj\merged_engineered_imputed.csv"   # update if file is elsewhere
DB_PATH  = r"C:\Users\HP\Desktop\suraj\gdf.db"
TABLE    = "final_file"                  # table name to create/replace

df = pd.read_csv(CSV_PATH)
with sqlite3.connect(DB_PATH) as conn:
    df.to_sql(TABLE, conn, if_exists="replace", index=False)

print(f"Wrote {len(df)} rows to {DB_PATH} -> table '{TABLE}'")


Wrote 2184 rows to C:\Users\HP\Desktop\suraj\gdf.db -> table 'final_file'
